# Анализ объявлений об аренде жилья

Имеется датасет со списком объявлений. В них так или иначе упоминается, какое количество людей желает арендовать жилье и на какой срок.
Данные неструктурированные.
И чтобы извлечь из них количество людей нам нужно написать промт и подключить LLM, которая поможет нам извлечь необходимые данные из неструктурированного текста.

Сначала установим необходимые пакеты:

In [17]:
!pip install langchain langchain-ollama langchain-community pandas tqdm -q


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


Мы будем использовать локальную LLM, поэтому для ее запуска не нужен ключ, но нужен Docker или локально установленная Ollama.
Для запуска модели в Docker:

In [19]:
!docker compose -f ./rental_ads/docker-compose.yml up -d

unable to get image 'ollama/ollama': Cannot connect to the Docker daemon at unix:///Users/aleksejkuzmin/.docker/run/docker.sock. Is the docker daemon running?


А для инициализации подключения к локально модели выполним следующее:

In [23]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model='gemma3:12b', # можно взять другую модель, но и результаты могут отличаться
                 base_url="http://localhost:11434/")

Теперь загрузим датасет с объявлениями

In [60]:
import pandas as pd
from tqdm import tqdm

df = pd.read_csv('./rental_ads/data/submission100lines.csv')

In [61]:
df.head()

,amount,text_id,text
0,1,14205200,Сниму жилье. 500-600 рублей сутки. Эконом клас...
1,5,319097075,ищем жилье в центре недалеко от моря с 23.07-0...
2,7,98881311,Ищем жилье на период с 18-28июля..на две семьи...
3,6,44587027,2 семьи по 3 человека (2 взрослых и ребенок) с...
4,6,352802829,Сниму недорогое жилье в лазаревском на 6 чел 3...


Шаблон промпта с инструкций, вопросом и выходным индикатором ниже.
В инструкции указываем LLM, что если она не знает ответа на наши вопрос, то пусть возвращает 0 людей.

In [62]:
prompt_template = """Ответь на вопрос, опираясь на контекст ниже.
Если на вопрос нельзя ответить, используя информацию из контекста,
ответь '0'. В контексте предоставлены заявки на аренду жилья, в которых упоминается количество человек, желающих снять жилье.

Context: {text_input}

Questions: Сколько человек планируют проживать в жилье в заявке, включая взрослых и детей?

Answer: Верни только одно число без какого-либо дополнительного текста.
"""

Дальше прогоняем каждую строку датасета через LLM используя промт выше

In [63]:
amount_list = [] # Список, где будем хранить ответы модели

for text_input in tqdm(df['text']):
    prompt = prompt_template.format(text_input=text_input) # Добавляем сообщение в промпт
    amount = llm.invoke(prompt).content # Ответ модели
    amount_list.append(amount) # Добавляем ответ в список

100%|██████████| 100/100 [02:20<00:00,  1.41s/it]


Очистим немного amount_list от мусора, некоторые числа не всегда все же как целые числа возвращаются.
И в целом ответ от LLM имеет тип str, а если мы захотим, например, сложить всех людей, то нужен сисловой тип int:

In [64]:
amount_list = [int(x.split('.')[0]) for x in amount_list]

Добавим эти числа как новый столбец к нашему датасету:

In [65]:
df['llm_answer'] = amount_list # Создаём новый столбец из ответов модели

Теперь мы можем посмотреть, что же там посчитала LLM'ка:

In [66]:
df.head()

,amount,text_id,text,llm_answer
0,1,14205200,Сниму жилье. 500-600 рублей сутки. Эконом клас...,1
1,5,319097075,ищем жилье в центре недалеко от моря с 23.07-0...,5
2,7,98881311,Ищем жилье на период с 18-28июля..на две семьи...,7
3,6,44587027,2 семьи по 3 человека (2 взрослых и ребенок) с...,6
4,6,352802829,Сниму недорогое жилье в лазаревском на 6 чел 3...,6


In [68]:
# Сравниваем и считаем совпадающие строки
matches = (df['amount'] == df['llm_answer']).sum()
print(f"Количество совпадающих строк: {matches}")

Количество совпадающих строк: 97


Посмотрим на те строки, где LLM посчитала неправильно:

In [69]:
# Находим строки, где столбцы НЕ совпадают
mismatched_rows = df[df['amount'] != df['llm_answer']]
wrong_answer = pd.DataFrame(mismatched_rows)
wrong_answer

,amount,text_id,text,llm_answer
27,3,112939206,"Муж, жена и 3,5 г. Ребенок,ищем жилье в Лазаре...",4
47,4,13769140,"Лазаревское, Солоники - бюджетное жилье на 3 с...",9
78,3,69438266,Здравствуйте ищем жилье в Сочи с 14.08 на 7 но...,4


Результаты могут отличать от запуска к запуску, но все равно процент совпадений довольно высокий - 97-98%

Ну и можем сохранить итоговый датасет в отдельный файл:

In [70]:
df.to_csv('./rental_ads/data/llm_counts_people.csv', index=False)